In [32]:
from PIL import Image, ImageDraw
import qrcode
import matplotlib.pyplot as plt
from tqdm import tqdm

In [33]:
def get_pixels_from_image(file_name='qrcode.png'):
    # Open the image
    img = Image.open(file_name)

    # Get the pixel data
    pixel_data = list(img.getdata())

    # Optionally, you can get the width and height of the image
    width, height = img.size

    return pixel_data, width, height

In [34]:
def generate_qr_code(url, file_name='qrcode.png', box_size = 11, border = 4, qr_version = 1):
    # Create QR code instance
    qr = qrcode.QRCode(
        version=qr_version,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=box_size,
        border=border,
    )

    # Add data to the QR code
    qr.add_data(url)
    qr.make(fit=True)

    # Create an image from the QR code instance
    img = qr.make_image(fill_color="black", back_color="white")

    # Save the image to a file
    img.save(file_name)

url_to_generate = 'https://www.ethermed.ai/'

qr_version = 1
box_size = 21
border = 4
generate_qr_code(url_to_generate, 'qrcode.png', box_size, border, qr_version)
print(f"QR code generated for {url_to_generate}")

QR code generated for https://www.ethermed.ai/


In [35]:
qr_code_file = 'qrcode.png'

# Get pixel data and image dimensions
pixels, img_width, img_height = get_pixels_from_image(qr_code_file)

pixels_01 = [['.','*'][i==255] for i in pixels]
# Print image dimensions
print("Image Dimensions:", img_width, "x", img_height)

Image Dimensions: 777 x 777


In [36]:
pixel_array = []
for i in range(0, len(pixels), img_height):
    row = pixels_01[i:i + img_height]
    # print(*row,sep='')
    pixel_array.append(row)

In [37]:
# for i in pixel_array:
#     print(*i,sep='')

In [38]:
def search_surroundings(pixel_array, i, j, distance = 1):
    result = []
    multiplier = distance*box_size
    for x_offset in [-multiplier, 0, multiplier]:
        for y_offset in [-multiplier, 0, multiplier]:
            if not (x_offset == 0 and y_offset == 0):
                one_pixel = pixel_array[i+y_offset][j+x_offset]
                if one_pixel == '.':
                    result.append((j+x_offset, i+y_offset))
    return result

In [39]:
def search_surrounding_tracks(track_addressed, i, j, distance = 1):
    result = []
    multiplier = distance*box_size
    for x_offset in [-multiplier, 0, multiplier]:
        for y_offset in [-multiplier, 0, multiplier]:
            if not (x_offset == 0 and y_offset == 0):
                tracked_piece = track_addressed[i+y_offset][j+x_offset]
                if tracked_piece > 0:
                    result.append((tracked_piece, j+x_offset, i+y_offset))
                     
    return result

In [40]:
def search_surrounding_tracks_lenient(track_addressed, pixel_array, i, j, distance = 1):
    result = []
    multiplier = distance*box_size
    for x_offset in [-multiplier, 0, multiplier]:
        for y_offset in [-multiplier, 0, multiplier]:
            if not (x_offset == 0 and y_offset == 0):
                tracked_piece = track_addressed[i+y_offset][j+x_offset]
                if tracked_piece > 0:
                    result.append((tracked_piece, j+x_offset, i+y_offset))

    if not result:
        for x_offset in [-multiplier, 0, multiplier]:
            for y_offset in [-multiplier, 0, multiplier]:
                if not (x_offset == 0 and y_offset == 0):
                    one_pixel = pixel_array[i+y_offset][j+x_offset]
                    if one_pixel == '.':
                        result.append((1, j+x_offset, i+y_offset))
                     
    return result

In [41]:
ethermed_color = (32,188,163)
ethermed_color_dark = (47,142,126)
ethermed_color_light = (79,204,180)

In [42]:
def draw_centered_cube(top_left_position, bottom_right_position, draw):
    top_left_x, top_left_y = top_left_position
    bottom_right_x, bottom_right_y = bottom_right_position
    y_dist = abs(bottom_right_y - top_left_y)

    mid_x = (top_left_x + bottom_right_x)//2
    top_y = top_left_y - y_dist//4
    middle_y = top_left_y + y_dist//4
    bottom_y = bottom_right_y + y_dist//4
    
    #Shrink vertically
    top_left_y += y_dist//8
    top_y += y_dist//8
    middle_y += y_dist//8
    bottom_y -= y_dist//8
    bottom_right_y -= y_dist//8
    
    poly_coords = [(bottom_right_x, bottom_right_y), (bottom_right_x, top_left_y), (mid_x, middle_y), (mid_x, bottom_y)]
    draw.polygon(poly_coords, fill=ethermed_color)
    
    poly_coords = [(bottom_right_x, top_left_y), (mid_x, top_y), (top_left_x, top_left_y), (mid_x, middle_y)]
    draw.polygon(poly_coords, fill=ethermed_color_light)
    
    poly_coords = [(mid_x, bottom_y), (mid_x, middle_y), (top_left_x, top_left_y), (top_left_x, bottom_right_y)]
    draw.polygon(poly_coords, fill=ethermed_color_dark)

    return draw

In [43]:
def draw_4_clock_cube(top_left_position, bottom_right_position, draw):
    x1, y1 = top_left_position
    x4, y4 = bottom_right_position
    x_dist = abs(x4 - x1)
    y_dist = abs(y4 - y1)
    x3 = x1 + 2*x_dist//3
    y3 = y1 + 2*y_dist//3
    x2 = x1 + x_dist//3
    y2 = y1 + y_dist//3

    poly_coords = [(x4, y4), (x4, y2), (x2, y2), (x2, y4)]
    draw.polygon(poly_coords, fill=ethermed_color)
    
    poly_coords = [(x1, y1), (x2, y2), (x2, y4), (x1, y3)]
    draw.polygon(poly_coords, fill=ethermed_color_dark)

    poly_coords = [(x1, y1), (x3, y1), (x4, y2), (x2, y2)]
    draw.polygon(poly_coords, fill=ethermed_color_light)
    return draw

In [44]:
def draw_4_clock_wireframe_cube(top_left_position, bottom_right_position, draw):
    x1, y1 = top_left_position
    x4, y4 = bottom_right_position
    x_dist = abs(x4 - x1)
    y_dist = abs(y4 - y1)
    x2 = x1 + x_dist//4
    x3 = x4 - x_dist//4
    y2 = y1 + y_dist//4
    y3 = y4 - y_dist//4

    width = 4
    #Back square
    draw.line([(x1, y1), (x1, y3)], fill=ethermed_color, width=width)
    draw.line([(x1, y3), (x3, y3)], fill=ethermed_color, width=width)
    draw.line([(x3, y3), (x3, y1)], fill=ethermed_color, width=width)
    draw.line([(x3, y1), (x1, y1)], fill=ethermed_color, width=width)

    #Front square
    draw.line([(x2, y2), (x2, y4)], fill=ethermed_color, width=width)
    draw.line([(x2, y4), (x4, y4)], fill=ethermed_color, width=width)
    draw.line([(x4, y4), (x4, y2)], fill=ethermed_color, width=width)
    draw.line([(x4, y2), (x2, y2)], fill=ethermed_color, width=width)

    #Connect lines
    draw.line([(x1, y1), (x2, y2)], fill=ethermed_color, width=width)
    draw.line([(x3, y1), (x4, y2)], fill=ethermed_color, width=width)
    draw.line([(x1, y3), (x2, y4)], fill=ethermed_color, width=width)
    draw.line([(x3, y3), (x4, y4)], fill=ethermed_color, width=width)
    
    return draw

In [45]:
def draw_8_clock_cube(top_left_position, bottom_right_position, draw):
    x1, y1 = top_left_position
    x4, y4 = bottom_right_position
    x_dist = abs(x4 - x1)
    y_dist = abs(y4 - y1)
    x3 = x1 + 2*x_dist//3
    y3 = y1 + 2*y_dist//3
    x2 = x1 + x_dist//3
    y2 = y1 + y_dist//3

    poly_coords = [(x1, y2), (x1, y4), (x3, y4), (x3, y2)]
    draw.polygon(poly_coords, fill=ethermed_color)
    
    poly_coords = [(x2, y1), (x1, y2), (x3, y2), (x4, y1)]
    draw.polygon(poly_coords, fill=ethermed_color_dark)

    poly_coords = [(x4, y1), (x4, y3), (x3, y4), (x3, y2)]
    draw.polygon(poly_coords, fill=ethermed_color_light)
    return draw

In [46]:
def draw_8_clock_wireframe_cube(top_left_position, bottom_right_position, draw):
    x1, y1 = top_left_position
    x4, y4 = bottom_right_position
    x_dist = abs(x4 - x1)
    y_dist = abs(y4 - y1)
    x2 = x1 + x_dist//4
    x3 = x4 - x_dist//4
    y2 = y1 + y_dist//4
    y3 = y4 - y_dist//4

    width = 4
    #Back square
    draw.line([(x2, y1), (x4, y1)], fill=ethermed_color, width=width)
    draw.line([(x4, y1), (x4, y3)], fill=ethermed_color, width=width)
    draw.line([(x4, y3), (x2, y3)], fill=ethermed_color, width=width)
    draw.line([(x2, y3), (x2, y1)], fill=ethermed_color, width=width)

    #Front square
    draw.line([(x1, y2), (x1, y4)], fill=ethermed_color, width=width)
    draw.line([(x1, y4), (x3, y4)], fill=ethermed_color, width=width)
    draw.line([(x3, y4), (x3, y2)], fill=ethermed_color, width=width)
    draw.line([(x3, y2), (x1, y2)], fill=ethermed_color, width=width)

    #Connect lines
    draw.line([(x4, y1), (x3, y2)], fill=ethermed_color, width=width)
    draw.line([(x2, y1), (x1, y2)], fill=ethermed_color, width=width)
    draw.line([(x4, y3), (x3, y4)], fill=ethermed_color, width=width)
    draw.line([(x2, y3), (x1, y4)], fill=ethermed_color, width=width)
    
    return draw

In [47]:
# width, height = img_width, img_height
# image = Image.new("RGB", (width, height), "white")
# draw = ImageDraw.Draw(image)

# top_left_position = (100, 100)
# bottom_right_position = (300, 300)
# draw_8_clock_wireframe_cube(top_left_position, bottom_right_position, draw)

# image.show()

In [48]:
def determine_cube_3x3(pixel_array, i, j, draw, track_addressed):
    base_pixel = pixel_array[i][j]
    pixel1 = pixel_array[i][j+box_size]
    pixel2 = pixel_array[i+box_size][j+box_size]
    pixel3 = pixel_array[i+box_size][j]
    pixel4 = pixel_array[i][j+box_size*2]
    pixel5 = pixel_array[i+box_size][j+box_size*2]
    pixel6 = pixel_array[i+box_size*2][j+box_size*2]
    pixel7 = pixel_array[i+box_size*2][j+box_size]
    pixel8 = pixel_array[i+box_size*2][j]
    
    rectangle_increment = 2
    if all(i=='.' for i in [base_pixel, pixel1, pixel2, pixel3, pixel5, pixel6, pixel7]) and all(i!='.' for i in [pixel4, pixel8]):
        draw_4_clock_cube((j+(box_size//2 - rectangle_increment),i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment)+box_size*2, i+(box_size//2 + rectangle_increment)+box_size*2), draw)
        
        track_addressed[i][j] = 3
        track_addressed[i][j+box_size] = 3
        track_addressed[i+box_size][j+box_size] = 3
        track_addressed[i+box_size][j] = 3
        track_addressed[i+box_size][j+box_size*2] = 3
        track_addressed[i+box_size*2][j+box_size*2] = 3
        track_addressed[i+box_size*2][j+box_size] = 3

    elif all(i=='.' for i in [base_pixel, pixel1, pixel2, pixel3, pixel4, pixel5, pixel7, pixel8]) and all(i!='.' for i in [base_pixel, pixel6]):
        draw_8_clock_cube((j+(box_size//2 - rectangle_increment),i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment)+box_size*2, i+(box_size//2 + rectangle_increment)+box_size*2), draw)
        
        track_addressed[i][j+box_size*2] = 3
        track_addressed[i][j+box_size] = 3
        track_addressed[i+box_size][j+box_size] = 3
        track_addressed[i+box_size][j] = 3
        track_addressed[i+box_size][j+box_size*2] = 3
        track_addressed[i+box_size*2][j] = 3
        track_addressed[i+box_size*2][j+box_size] = 3
    
    elif all(i=='.' for i in [base_pixel, pixel1, pixel3, pixel5, pixel6, pixel7]) and all(i!='.' for i in [pixel2, pixel4, pixel8]):
        draw_4_clock_wireframe_cube((j+(box_size//2 - rectangle_increment),i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment)+box_size*2, i+(box_size//2 + rectangle_increment)+box_size*2), draw)
        
        track_addressed[i][j] = 3
        track_addressed[i][j+box_size] = 3
        track_addressed[i+box_size][j+box_size] = 3
        track_addressed[i+box_size][j] = 3
        track_addressed[i+box_size][j+box_size*2] = 3
        track_addressed[i+box_size*2][j+box_size*2] = 3
        track_addressed[i+box_size*2][j+box_size] = 3

    
    elif all(i=='.' for i in [base_pixel, pixel1, pixel3, pixel4, pixel5, pixel7, pixel8]) and all(i!='.' for i in [base_pixel, pixel2, pixel6]):
        draw_8_clock_wireframe_cube((j+(box_size//2 - rectangle_increment),i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment)+box_size*2, i+(box_size//2 + rectangle_increment)+box_size*2), draw)
        
        track_addressed[i][j+box_size*2] = 3
        track_addressed[i][j+box_size] = 3
        track_addressed[i+box_size][j+box_size] = 3
        track_addressed[i+box_size][j] = 3
        track_addressed[i+box_size][j+box_size*2] = 3
        track_addressed[i+box_size*2][j] = 3
        track_addressed[i+box_size*2][j+box_size] = 3

    return track_addressed


In [49]:
def determine_cube_2x2(pixel_array, i, j, draw, track_addressed):
    base_pixel = pixel_array[i][j]
    pixel1 = pixel_array[i][j+box_size]
    pixel2 = pixel_array[i+box_size][j+box_size]
    pixel3 = pixel_array[i+box_size][j]
    
    rectangle_increment = 2
    if all(i=='.' for i in [base_pixel, pixel1, pixel2, pixel3]):
        draw_centered_cube((j+(box_size//2 - rectangle_increment),i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment)+box_size, i+(box_size//2 + rectangle_increment)+box_size), draw)
        
        for y_offset in [0, 1]:
            for x_offset in [0, 1]:
                track_addressed[i+y_offset*box_size][j+x_offset*box_size] = 2

    return track_addressed

In [50]:
width, height = img_width, img_height

In [51]:
image = Image.new("RGB", (width, height), "white")
draw = ImageDraw.Draw(image)

alternate = 0
corner_boxes = (8+border)*box_size

track_addressed = [[0 for _ in range(width)] for _ in range(height)]

#Fill all 3x3 shapes
for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
    for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        if not ((0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes)):
            """
            b14
            325
            876
            """
            base_pixel = pixel_array[i][j]
            pixel1 = pixel_array[i][j+box_size]
            pixel2 = pixel_array[i+box_size][j+box_size]
            pixel3 = pixel_array[i+box_size][j]
            pixel4 = pixel_array[i][j+box_size*2]
            pixel5 = pixel_array[i+box_size][j+box_size*2]
            pixel6 = pixel_array[i+box_size*2][j+box_size*2]
            pixel7 = pixel_array[i+box_size*2][j+box_size]
            pixel8 = pixel_array[i+box_size*2][j]

            track_addressed = determine_cube_3x3(pixel_array, i, j, draw, track_addressed)

#Fill all 2x2 shapes
for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
    for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        if not ((0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes)):
            """
            b1
            32
            """
            #Don't draw 2x2 shapes over 3x3 shapes
            if any(track_addressed[i+y_offset*box_size][j+x_offset*box_size] > 0  for y_offset in [0,1] for x_offset in [0,1]):
                continue

            track_addressed = determine_cube_2x2(pixel_array, i, j, draw, track_addressed)

#Draw little squares to help qr code detection
for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
    for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        one_pixel = pixel_array[i][j]
        if one_pixel == '.':
            if (0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes):
                rectangle_coords = [(j, i), (j+box_size, i+box_size)]
                draw.rectangle(rectangle_coords, fill=ethermed_color)
            else:
                if track_addressed[i][j] > 0:
                    continue
                
                rectangle_increment = 3
                rectangle_coords = [(j+(box_size//2 - rectangle_increment), i+(box_size//2 - rectangle_increment)), (j+(box_size//2 + rectangle_increment), i+(box_size//2 + rectangle_increment))]
                draw.rectangle(rectangle_coords, fill=ethermed_color)

                #Testing triangles in grid area. Alternating doesn't work for some reason but one or other works just fine. 
                # if alternate:
                #     triangle_coords = [(j+1, i+1), (j+9, i+1), (j+1, i+9)] #x, y going down
                #     draw.polygon(triangle_coords, fill=ethermed_color)
                # else:
                #     triangle_coords = [(j+9, i+1), (j+1, i+9), (j+9, i+9)]
                #     draw.polygon(triangle_coords, fill=ethermed_color)
                # alternate = not alternate   
        #Draw borders to verify open spaces
        # else:
        #     if not (0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes):
        #         rectangle_coords = [(j, i), (j+box_size, i+box_size)]
        #         draw.rectangle(rectangle_coords, fill='white', outline = ethermed_color_light, width=1)

#Track last cells
endpoints = [[0 for _ in range(width)] for _ in range(height)]

#Connect lines
change = 1
while change:
    change = 0
    for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
            if (0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes):
                continue
            
            one_pixel = pixel_array[i][j]
            addressed = track_addressed[i][j]
            if one_pixel == '.' and addressed < 1:
                surroundings = search_surrounding_tracks(track_addressed, i, j, 1)
                surroundings.sort(key=lambda x: x[0], reverse=True)

                if surroundings:
                    track_value, x_destination, y_destination = surroundings[0]
                    draw.line([(j+box_size//2, i+box_size//2), (x_destination+box_size//2, y_destination+box_size//2)], fill=ethermed_color, width=4)
                    track_addressed[i][j] = track_value
                    endpoints[i][j] = 1
                    endpoints[y_destination][x_destination] = 0
                    change = 1

#Connect lines
#more lenient
change = 1
while change:
    change = 0
    for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
            if (0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes):
                continue
            
            one_pixel = pixel_array[i][j]
            addressed = track_addressed[i][j]
            if one_pixel == '.' and addressed < 1:
                surroundings = search_surrounding_tracks_lenient(track_addressed, pixel_array, i, j, 1)
                surroundings.sort(key=lambda x: x[0], reverse=True)

                if surroundings:
                    track_value, x_destination, y_destination = surroundings[0]
                    draw.line([(j+box_size//2, i+box_size//2), (x_destination+box_size//2, y_destination+box_size//2)], fill=ethermed_color, width=4)
                    track_addressed[i][j] = track_value
                    endpoints[i][j] = 1
                    endpoints[y_destination][x_destination] = 0
                    change = 1

#Draw boxes for endpoints
for i in range(box_size*border, len(pixel_array)-box_size*border, box_size):
        for j in range(box_size*border, len(pixel_array)-box_size*border, box_size):
            if (0<i<corner_boxes and 0<j<corner_boxes) or (0<i<corner_boxes and (height-corner_boxes)<j<height) or ((height-corner_boxes)<i<height and 0<j<corner_boxes):
                continue
            
            endpoint = endpoints[i][j]
            if endpoint == 1:
                draw_centered_cube((j,i), (j+box_size, i+box_size), draw)

            one_pixel = pixel_array[i][j]
            addressed = track_addressed[i][j]
            if one_pixel == '.' and addressed < 1:
                draw_centered_cube((j,i), (j+box_size, i+box_size), draw)
image.show()
